1. Read the file (using readlines), store the sequence of its lines in a variable.
2. Remove the '\n' symbol in the end of each line.
3. Create a variable for counting sequences and set it to zero.
4. Create a list for storing the features.
5. Iterate over the elements of the sequence (using enumerate). The enumerate function will provide the number of each element in addition to the element itself. For each element:
   
   if the element is an empty string:
       increase the sentence counter
   else:
       + a) split the element into the word and the POS tag;
       + b) determine if the element is the first in the sequence or the first in the sentence (i.e. the preceeding element is an empty string);
       + c) access the next element in the sequence (by its number);
       + d) split it into the word itself and the POS tag;
       + e) determine if its the last element in the sequence or the last element in the sentence (i.e. the succeeding element is an empty string);
       + f) create the following tuple:
          (sentence_no, token_1, token_2, POS_1, POS_2, the indicator from b, the indicator from e);
       + g) append the tuple to the list you created in 4.
6. Turn the list with the features into a table (I will show you how).

In [19]:
file = open("../Assignment 2/wsj_tagged/wsj_tagged_train.tt")
line_sequence = file.readlines()

In [41]:
# TODO: handle the dots!

def features_extractor(line_sequence):
    features = [] #Create a list for storing the features.
    sentence_counter = 0 #Create a variable for counting sequences and set it to zero.
    
    stripped_lines = []
    for line in line_sequence:
        stripped_lines.append(line.strip()) #Remove the '\n' symbol in the end of each line.
        
    for (i, line) in enumerate(stripped_lines): #The enumerate function will provide the number of each element in addition to the element itself.     
        if i == (len(stripped_lines) - 1) or stripped_lines[i+1] == "":
            continue
        if line == "":
            sentence_counter += 1
        else: #split the element into the word and the POS tag
            splitted_line1 = line.split("\t") # ("I", "PN"/ "have", "V" ...)
            word1 = splitted_line1[0]
            POS_tag1 = splitted_line1[1] 
            
            if i == 0: # i is a number of a line
                first_in_sent = True
            elif stripped_lines[i-1] == "":
                first_in_sent = True
            else:
                first_in_sent = False
                
            if (i+1) == (len(stripped_lines) - 1):
                last_in_sent = True
            elif stripped_lines[i+2] == "":
                last_in_sent = True
            else:
                last_in_sent = False
            
            splitted_line2 = stripped_lines[i + 1].split("\t") #(I have/ have a/ a pen ...)
            word2 = splitted_line2[0]
            POS_tag2 = splitted_line2[1]
            
            features.append((sentence_counter, word1, word2, POS_tag1, POS_tag2, first_in_sent, last_in_sent))
    
    return features

In [45]:
import pandas as pd

In [48]:
features_df = pd.DataFrame(features_extractor(line_sequence),
                           columns=["sent_no",
                                    "word1",
                                    "word2",
                                    "POS_Tag1",
                                    "POS_tag2",
                                    "first_in_sent",
                                    "last_in_sent"])

In [51]:
features_df.to_csv("features.csv", index=False)

In [240]:
file_name = "wsj_0600"

In [241]:
RST_file = open("../nlp_project/rst_discourse_treebank/data/RSTtrees-WSJ-main-1.0/TRAINING/" + file_name + ".out.edus")
lines = RST_file.readlines()
RST_file.close()

In [26]:
print(lines)

['    Spencer J. Volk, president and chief operating officer of this consumer and industrial products company, was elected a director. \n', '    Mr. Volk, 55 years old, succeeds Duncan Dwight, \n', '    who retired in September. \n']


In [32]:
# tokenizing the EDUs
from nltk import word_tokenize

tokenized_edus = []
for edu in lines:
    tokenized_edu = word_tokenize(edu)
    tokenized_edus.append(tokenized_edu)
print(tokenized_edus)

[['Spencer', 'J.', 'Volk', ',', 'president', 'and', 'chief', 'operating', 'officer', 'of', 'this', 'consumer', 'and', 'industrial', 'products', 'company', ',', 'was', 'elected', 'a', 'director', '.'], ['Mr.', 'Volk', ',', '55', 'years', 'old', ',', 'succeeds', 'Duncan', 'Dwight', ','], ['who', 'retired', 'in', 'September', '.']]


In [185]:
# Remember boundary indices, combine EDUs of a sentence
boundary_indices = [] 
edu_boundary = []
sentences = []
sentence = []
for edu in tokenized_edus:
    if edu[-1] not in ["!", "?", ".", "..."]:
        sentence.extend(edu)
        edu_boundary.append(len(sentence) - 1)
    else:
        sentence.extend(edu)
        sentences.append(sentence)
        edu_boundary.append(len(sentence) - 1)
        boundary_indices.append(edu_boundary)
        sentence = []
        edu_boundary = []

In [186]:
print(sentence)

[]


In [187]:
print(sentences)

[['Spencer', 'J.', 'Volk', ',', 'president', 'and', 'chief', 'operating', 'officer', 'of', 'this', 'consumer', 'and', 'industrial', 'products', 'company', ',', 'was', 'elected', 'a', 'director', '.'], ['Mr.', 'Volk', ',', '55', 'years', 'old', ',', 'succeeds', 'Duncan', 'Dwight', ',', 'who', 'retired', 'in', 'September', '.']]


In [188]:
print(edu_boundary)

[]


In [189]:
print(edu_boundaries)

[21, 10, 15]


### POS Tagging

In [245]:
# getting a POS-tag from nltk
pos_sentences = []
for sentence in sentences:
    pos_sentence = nltk.pos_tag(sentence)
    pos_sentences.append(pos_sentence)

In [246]:
print(pos_sentence)

[('Mr.', 'NNP'), ('Volk', 'NNP'), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('succeeds', 'VBZ'), ('Duncan', 'NNP'), ('Dwight', 'NNP'), (',', ','), ('who', 'WP'), ('retired', 'VBD'), ('in', 'IN'), ('September', 'NNP'), ('.', '.')]


### Word2Vec Embedding

In [197]:
!pip install gensim

    100% |████████████████████████████████| 22.3MB 47kB/s eta 0:00:011  34% |███████████                     | 7.6MB 5.3MB/s eta 0:00:03    39% |████████████▋                   | 8.8MB 34.6MB/s eta 0:00:01    61% |███████████████████▋            | 13.7MB 6.0MB/s eta 0:00:02    68% |██████████████████████          | 15.3MB 4.3MB/s eta 0:00:02
    100% |████████████████████████████████| 133kB 4.0MB/s ta 0:00:01
    100% |████████████████████████████████| 4.1MB 257kB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 4.8MB/s ta 0:00:011
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /Users/YK/Library/Caches/pip/wheels/36/48/35/97efc2bd1b233627131c9a936c9de23681846db707b907d353
  Running setup.py bdist_wheel for bz2file ... done
  Stored in directory: /Users/YK/Library/Caches/pip/wheels/31/9c/20/996d65ca104cbca940b1b053299b68459391c01c774d073126
Successfully built smart-open bz2file


In [202]:
# to train word2vec, doesn't give vecs for POS

import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [227]:
word_vectors = []
for sentence in sentences:
    for word in sentence:
        if word in model.vocab:
            vector = model.get_vector(word)
            word_vectors.append(vector)
        else:
            word_vectors.append(None)

In [228]:
print(word_vectors)

[array([-0.0039978 ,  0.00714111, -0.31054688,  0.01464844,  0.1015625 ,
       -0.19921875, -0.05200195, -0.11816406, -0.16992188, -0.0213623 ,
        0.09521484, -0.16210938, -0.06396484,  0.1171875 , -0.01025391,
       -0.07958984,  0.12988281,  0.09716797,  0.06640625,  0.11279297,
        0.12109375, -0.16796875,  0.01867676, -0.07617188,  0.06689453,
        0.06542969, -0.18945312,  0.12890625,  0.21972656,  0.09765625,
       -0.0546875 , -0.17480469, -0.19824219, -0.21484375, -0.07421875,
        0.04638672,  0.2265625 ,  0.11425781, -0.05078125, -0.20410156,
       -0.07373047, -0.19335938, -0.08886719,  0.03686523,  0.10009766,
        0.00524902, -0.11572266,  0.02209473,  0.17578125, -0.07958984,
        0.20019531, -0.00558472, -0.16503906, -0.21679688,  0.06835938,
       -0.11181641,  0.02624512, -0.13671875, -0.02478027, -0.16113281,
       -0.08349609, -0.0546875 , -0.22949219, -0.04663086, -0.11572266,
       -0.12890625, -0.14257812,  0.13476562,  0.06176758, -0.0

In [236]:
!./stanford-parser-full-2017-06-09/lexparser.sh parsing-eval/wsj.22.mrg.text > englishRNN.txt

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
Exception in thread "main" edu.stanford.nlp.io.RuntimeIOException: edu.stanford.nlp.process.DocumentPreprocessor: Could not open path parsing-eval/wsj.22.mrg.text
	at edu.stanford.nlp.process.DocumentPreprocessor.<init>(DocumentPreprocessor.java:137)
	at edu.stanford.nlp.parser.lexparser.ParseFiles.parseFiles(ParseFiles.java:136)
	at edu.stanford.nlp.parser.lexparser.ParseFiles.parseFiles(ParseFiles.java:75)
	at edu.stanford.nlp.parser.lexparser.LexicalizedParser.main(LexicalizedParser.java:1517)
Caused by: java.io.IOException: Unable to open "parsing-eval/wsj.22.mrg.text" as class path, filename or URL
	at edu.stanford.nlp.io.IOUtils.getInputStreamFromURLOrClasspathOrFileSystem(IOUtils.java:480)
	at edu.stanford.nlp.io.IOUtils.readerFromString(IOUtils.java:637)
	at edu.stanford.

In [254]:
# writing a text in conll format
conll_file = open("edu_segmentation/" + file_name + ".conll", "w")
 
for pos_sentence in pos_sentences:
    for word, pos in pos_sentence:
        conll_file.write(word + " " + pos + "\n")
    conll_file.write("\n")
    
conll_file.close()

In [273]:
with open("edu_segmentation/wsj_0600.conll") as file:
    print(file.read())

Spencer NNP
J. NNP
Volk NNP
, ,
president NN
and CC
chief NN
operating VBG
officer NN
of IN
this DT
consumer NN
and CC
industrial JJ
products NNS
company NN
, ,
was VBD
elected VBN
a DT
director NN
. .

Mr. NNP
Volk NNP
, ,
55 CD
years NNS
old JJ
, ,
succeeds VBZ
Duncan NNP
Dwight NNP
, ,
who WP
retired VBD
in IN
September NNP
. .




In [262]:
import os
os.environ["FILE_NAME"] = file_name

In [274]:
! ./stanford-parser-full-2017-06-09/lexparser.sh edu_segmentation/wsj_0600.conll > edu_segmentation/"$FILE_NAME".penn 2> /dev/null

In [ ]:
1. Read the penn bank 
# for every tree use 

In [282]:
with open("edu_segmentation/wsj_0600.penn") as file:
    trees_string = file.read()
    trees = trees_string.split("\n\n")

In [284]:
# constructing the tree: string to sequence of its elements
def split(tree_str):
    chunks = tree_str.split()
    chunk_elements = []
    for chunk in chunks:
        if chunk[0] == "(":
            index_i = 0
            while chunk[index_i] == "(":
                chunk_elements.append("(")
                index_i += 1
            chunk_elements.append(chunk[index_i:])
        else:
            index_j = (-1)
            while chunk[index_j] == ")":
                chunk_elements.append(")")
                index_j += (-1)
            chunk_elements.insert((index_j + 1), chunk[0:((index_j)+1)])
    return(chunk_elements)

In [285]:
# generating constituents
def constituents(tree_str):
# nt = non-terminal
    nt_seq = []
    term_counter = 0
    constituents_set = set()
    parsed_str = split(tree_str)
    element_index = 0
    none_depth = 0
    while element_index < len(parsed_str):
        if parsed_str[element_index] == "(":
            nt = parsed_str[element_index + 1]
            if nt == "-NONE-":
# we are entering a subtree beginning with -NONE-                
                none_depth += 1
                nt_seq.append((nt, term_counter + 1))
            else:
# taking first of the nt-label
                if len(nt) > 0 and nt[0] == "-":
                    first_nt_label = nt
                else:
                    first_nt_label = nt.split("-")[0]
                nt_seq.append((first_nt_label, term_counter + 1))
#  skipping a non-terminal           
            element_index += 2
        elif parsed_str[element_index] == ")":
            last_nt = nt_seq.pop()
            if last_nt[0] == "-NONE-":
# exiting a subtree beginning with -NONE-                
                none_depth -= 1
            else:
                if last_nt[1] <= term_counter:
                    if last_nt[0] == "ROOT" or last_nt[0] =="TOP":
                        pass
                    else:
                        constituents_set.add((last_nt[0], last_nt[1], term_counter))
                else:
# In this case all terminals in the subtree were ignored, which means that the subtree contained nothing but traces, 
# therefore we're skipping the constituent.
                    pass
            element_index += 1
        else:
            if none_depth == 0:
# if we are not in a subtree beginning with -NONE-, then we count the terminal                
                term_counter += 1
            else:
# otherwise we ignore the terminal            
                pass
            element_index += 1
    return constituents_set


In [286]:
print(constituents(trees[0]))

{('S', 1, 44), ('VBP', 30, 30), ('NNP', 2, 2), ('NNP', 6, 6), ('CC', 11, 11), ('NNP', 18, 18), ('NNP', 22, 22), ('NN', 10, 10), ('NNP', 26, 26), ('NNP', 40, 40), ('NN', 15, 15), ('NN', 23, 23), ('NN', 31, 31), ('NN', 41, 41), ('NNP', 1, 1), ('IN', 19, 19), ('NP', 39, 43), ('JJ', 13, 13), ('NP', 42, 43), ('S', 8, 33), ('NP', 39, 41), ('ADVP', 36, 36), ('PRN', 7, 34), ('NP', 9, 18), ('NP', 38, 38), ('.', 43, 43), ('NNS', 32, 32), ('NP', 21, 29), (',', 7, 7), (',', 33, 33), ('NNP', 4, 4), ('NNP', 12, 12), ('NNP', 16, 16), ('NNP', 24, 24), ('CC', 25, 25), ('RB', 36, 36), ('VBN', 37, 37), ('NNP', 38, 38), ('NNP', 42, 42), ('S', 38, 43), ('JJ', 27, 27), ('NN', 9, 9), ('NN', 17, 17), ('VP', 37, 43), (',', 8, 8), ('.', 44, 44), ('NP', 9, 29), ('VBD', 35, 35), ('NN', 28, 28), ('NNP', 5, 5), ('PP', 20, 29), ('NNS', 29, 29), ('VP', 35, 43), ('DT', 21, 21), ('NP', 1, 6), ('DT', 39, 39), ('IN', 20, 20), ('NNP', 3, 3), ('VP', 30, 32), ('NP', 31, 32), (',', 34, 34), ('PP', 19, 29), ('NN', 14, 14)}


In [ ]:
one_word_constituents = []
for tree in trees:
    tree_dict = {}
    for constituent in constituents(tree):
        if constituent[1] not in tree_dict:
            tree_dict[constituent[1]] = ([],[])
            
        # add constituents[1] to [0] element of a dict 
        # add constituents[2] to a word in a list which has the same word at this index?
    

In [98]:
import nltk

In [14]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True